In [1]:
library("tidyverse")
library("data.table")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




# Fish numbers

In [2]:
df <- fread("/lts/research/birney/users/saul/nextflow/medaka_behaviour_pilot/hmm/time_step0.08_n_states14_hmm.csv.gz")

In [4]:
ids <- df[, unique(id)]

In [29]:
pairs <- str_remove(ids, "_q.") |> str_remove("_(L|R)") |> str_remove("^[0-9]*_[0-9]*_") |> table()

In [30]:
pairs


  icab_hdr_no_ref  icab_hdr_no_test   icab_hdr_of_ref  icab_hdr_of_test 
               60                60                60                60 
  icab_hni_no_ref  icab_hni_no_test   icab_hni_of_ref  icab_hni_of_test 
               76                76                76                76 
  icab_ho5_no_ref  icab_ho5_no_test   icab_ho5_of_ref  icab_ho5_of_test 
               56                56                56                56 
 icab_icab_no_ref icab_icab_no_test  icab_icab_of_ref icab_icab_of_test 
               68                68                68                68 
 icab_kaga_no_ref icab_kaga_no_test  icab_kaga_of_ref icab_kaga_of_test 
               47                47                47                47 

In [31]:
pairs[grepl("of_ref", names(pairs))]


 icab_hdr_of_ref  icab_hni_of_ref  icab_ho5_of_ref icab_icab_of_ref 
              60               76               56               68 
icab_kaga_of_ref 
              47 

# Tracking accuracy

In [2]:
f_list <- list.files("/lts/research/birney/users/saul/nextflow/medaka_behaviour_pilot/trajectories_csv/", full.names = TRUE)

In [3]:
df <- lapply(
    f_list,
    function(f){
        fread(f)[, id := str_remove(basename(f), "_traj_(with_identities)?.csv.gz")]
    }
) |> rbindlist(use.names = TRUE, fill = TRUE)

In [4]:
df <- df[, .(frame = 1:.N, ref_x, ref_y, test_x, test_y), by = id]

In [5]:
df <- melt(df, id.vars = c("id", "frame"))

In [6]:
df[, fish := str_remove(variable, "_(x|y)")]
df[, variable := str_remove(variable, "(ref|test)_")]

In [7]:
df <- dcast(df, id + frame + fish ~ variable, value.var = "value")

In [8]:
summary <- df[, .(tracked_fraction = mean(!is.na(x) & !is.na(y))), by = c("id", "fish")]
remove(df)

In [9]:
summary[, min(tracked_fraction)]

[1] 0.7800556

In [11]:
summary[, mean(tracked_fraction >= 0.99)]

[1] 0.8802932